In [1]:
#text_file = "eng-ban.txt"
text_file = "ban-eng.txt"
# To open the text file we need to encode the text. Here, we use 'utf8' encoding
with open(text_file, encoding="utf8") as f:
    lines = f.read().split("\n")[:-1]
text_pairs = []
for line in lines:
    bangla, english = line.split(" >>> ")
    bangla = "[start] " + bangla + " [end]"
    text_pairs.append((english, bangla))

In [2]:
print("Total Sentences:", len(text_pairs))
#text_pairs

Total Sentences: 20


In [5]:
import random
for i in range(5):
    print(random.choice(text_pairs));

('Nope.', '[start] না. [end]')
('About hundrad million people speaks in Bangla.', '[start] প্রায় এক কোটি মানুষ বাংলায় কথা বলে. [end]')
('Sorry.', '[start] দুঃখিত. [end]')
('Bangladesh is a developing country.', '[start] বাংলাদেশ একটি উন্নয়নশীল দেশ. [end]')
('It is a Bangla to English translation file.', '[start] এটা বাংলা থেকে ইংরেজি অনুবাদের নথি. [end]')


In [13]:
import random
random.shuffle(text_pairs)
total_data_length = len(text_pairs)
num_val_samples = int(0.20 * total_data_length)

In [14]:
print(total_data_length)
print(num_val_samples)

20
4


In [15]:
num_train_samples = (total_data_length - 2 * num_val_samples)
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]

In [17]:
print(f"Number of Train Pairs: {len(train_pairs)}")
print(f"Number of Val Pairs: {len(val_pairs)}")
print(f"Number of Test Pairs: {len(test_pairs)}")

12
[('I know Bangla.', '[start] আমি বাংলা জানি. [end]'), ('Are you mad!', '[start] তুমি কি পাগল! [end]'), ('About seventy percentage of people are literate.', '[start] বাংলাদেশের প্রায় সত্তর ভাগ মানুষ স্বাক্ষর. [end]'), ('No.', '[start] না. [end]'), ('Bangladesh is a south-asian country.', '[start] বাংলাদেশ দক্ষিন এশিয়ার একটি দেশ. [end]'), ('Sorry.', '[start] দুঃখিত. [end]'), ('Nope.', '[start] না. [end]'), ('Bangladesh is a developing country.', '[start] বাংলাদেশ একটি উন্নয়নশীল দেশ. [end]'), ("I don't know Bangla.", '[start] আমি বাংলা জানি না. [end]'), ('About hundrad million people speaks in Bangla.', '[start] প্রায় এক কোটি মানুষ বাংলায় কথা বলে. [end]'), ('Allas! We lost the game.', '[start] হায়! আমরা খেলায় হেরেছি. [end]'), ('It is a Bangla to English translation file.', '[start] এটা বাংলা থেকে ইংরেজি অনুবাদের নথি. [end]')]
4 
 [('I sing song in Bangla.', '[start] আমি বাংলায় গান গাই. [end]'), ('21st of February is the International Mother Language Day.', '[start] একুশে ফেব্রুয়ারি আন্

In [74]:
print(test_pairs)

('About seventy percentage of people are literate.', '[start] বাংলাদেশের প্রায় সত্তর ভাগ মানুষ স্বাক্ষর. [end]')


In [5]:
import tensorflow as tf
import string
import re

strip_chars = string.punctuation + ""
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

In [7]:
f"[{re.escape(strip_chars)}]"

'[!"\\#\\$%\\&\'\\(\\)\\*\\+,\\-\\./:;<=>\\?@\\\\\\^_`\\{\\|\\}\\~\\?]'

In [6]:
# XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
from tensorflow.keras import layers

def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return(tf.strings.regex_replace(
        lowercase, f"[{re.escape(strip_chars)}]", ""
    ))

vocab_size = 10000
sequence_length = 20
source_vectorization = layers.TextVectorization(
    max_tokens = vocab_size,
    output_mode = "int",
    output_sequence_length = sequence_length
)
target_vectorization = layers.TextVectorization(
    max_tokens = vocab_size,
    output_mode = "int",
    output_sequence_length = sequence_length + 1,
    standardize = custom_standardization
)

In [7]:
train_english_texts = [pair[0] for pair in train_pairs]
train_bangla_texts = [pair[1] for pair in train_pairs]
source_vectorization.adapt(train_english_texts)
target_vectorization.adapt(train_bangla_texts)

In [8]:
batch_size = 64

def format_dataset(eng, ban):
    eng = source_vectorization(eng)
    ban = source_vectorization(ban)
    return({"english": eng, "bangla": ban[:, :-1]}, ban[:, 1:])

def make_dataset(pairs):
    eng_texts, ban_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    ban_texts = list(ban_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, ban_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls = 4)
    return(dataset.shuffle(2048).prefetch(16).cache())

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [22]:
#print(list(train_ds.as_numpy_iterator()));

In [9]:
for inputs, targets in train_ds.take(1):
    print(f"inputs['english'].shape: {inputs['english'].shape}")
    print(f"inputs['bangla'].shape: {inputs['bangla'].shape}")
    print(f"target.shape: {targets.shape}")

inputs['english'].shape: (12, 20)
inputs['bangla'].shape: (12, 19)
target.shape: (12, 19)


In [10]:
from tensorflow import keras

embed_dim = 256
latent_dim = 1024

In [11]:
source = keras.Input(shape=(None,), dtype="int64", name="english")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(source)
encoded_source = layers.Bidirectional(layers.GRU(latent_dim), merge_mode="sum")(x)

In [12]:
past_target = keras.Input(shape=(None,), dtype="int64", name="bangla")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(past_target)
decoder_gru = layers.GRU(latent_dim, return_sequences=True)
x = decoder_gru(x, initial_state=encoded_source)
x = layers.Dropout(0.5)(x)
target_next_step = layers.Dense(vocab_size, activation="softmax")(x)
seq2seq_rnn = keras.Model([source, past_target], target_next_step)

In [29]:
#seq2seq_rnn.summary()

In [13]:
seq2seq_rnn.compile(
    optimizer = "rmsprop",
    loss = "sparse_categorical_crossentropy",
    metrics = ["accuracy"]
)
seq2seq_rnn.fit(train_ds, epochs=50, validation_data=val_ds)

Epoch 1/50
1/1 [==============================] - 26s 26s/step - loss: 2.7881 - accuracy: 0.0000e+00 - val_loss: 2.7980 - val_accuracy: 0.8333
Epoch 2/50
1/1 [==============================] - 2s 2s/step - loss: 2.6784 - accuracy: 0.8261 - val_loss: 1.6233 - val_accuracy: 0.8333
Epoch 3/50
1/1 [==============================] - 2s 2s/step - loss: 1.4187 - accuracy: 0.8261 - val_loss: 0.2749 - val_accuracy: 0.8333
Epoch 4/50
1/1 [==============================] - 2s 2s/step - loss: 0.4251 - accuracy: 0.8261 - val_loss: 0.2723 - val_accuracy: 0.3333
Epoch 5/50
1/1 [==============================] - 2s 2s/step - loss: 0.4180 - accuracy: 0.4058 - val_loss: 0.2750 - val_accuracy: 0.8333
Epoch 6/50
1/1 [==============================] - 3s 3s/step - loss: 0.3662 - accuracy: 0.8261 - val_loss: 0.1781 - val_accuracy: 0.8333
Epoch 7/50
1/1 [==============================] - 3s 3s/step - loss: 0.2637 - accuracy: 0.8261 - val_loss: 0.1389 - val_accuracy: 0.8333
Epoch 8/50
1/1 [===================

In [14]:
import numpy as np
ban_vocab = target_vectorization.get_vocabulary()
ban_index_lookup = dict(zip(range(len(ban_vocab)), ban_vocab))
max_decoded_sentence_length = 20

In [20]:
# English texts
train_eng_texts = [pair[0] for pair in train_pairs]
print(f"Train Eng Text: {len(train_eng_texts)}")

test_eng_texts = [pair[0] for pair in test_pairs]
print(f"Test Eng Text: {len(test_eng_texts)}")

val_eng_texts = [pair[0] for pair in val_pairs]
print(f"Val Eng Text: {len(val_eng_texts)}")

# Bangla texts
train_ban_texts = [pair[1] for pair in train_pairs]
print(f"\nTrain Ban Text: {len(train_ban_texts)}")

test_ban_texts = [pair[1] for pair in test_pairs]
print(f"Test Ban Text: {len(test_ban_texts)}")

val_ban_texts = [pair[1] for pair in val_pairs]
print(f"Val Ban Text: {len(val_ban_texts)}")

Train Eng Text: 12
Test Eng Text: 2
Val Eng Text: 2

Train Ban Text: 12
Test Ban Text: 2
Val Ban Text: 2


In [57]:
input_sentence = random.choice(test_eng_texts)
print(input_sentence)

[
